# Qui va sobreviure al Titanic?

En aquest exercici, utilitzarem les dades del Titanic per predir si un passatger va sobreviure o no al naufragi. 
Veurem diferent formes de fer-ho; amb diversos graus d'automatisme.

## Human learn

Human learn és una llibreria que ens permet combinar el machine learning amb el coneixement humà de forma senzilla.

In [ ]:
# Carreguem les llibreries i les dades

In [ ]:
# Instal·lem la llibreria
%pip install git+https://github.com/koaning/human-learn.git

In [ ]:
from hulearn.datasets import load_titanic

In [ ]:
df = load_titanic(as_frame=True)
df

In [ ]:
from sklearn.model_selection import train_test_split

# Preparem les dades: separem les variables explicatives de la variable objectiu i creem els conjunts d'entrenament i de test
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['name', 'survived'], axis=1), df['survived'], test_size=0.25, random_state=42)

### Classificació amb una funció

En aquest primer exemple, utilitzarem una funció per classificar els passatgers. Es tracta d'una funció molt senzilla que classifica els passatgers en funció del preu del bitllet. El plantejament és que els passatgers que van pagar més tenien més probabilitats de sobreviure.

In [ ]:
# FunctionClassifier ens permet crear un classificador a partir d'una funció
from hulearn.classification import FunctionClassifier

In [ ]:
# Utilitzarem el FunctionClassifier per crear un classificador que utilitzi el preu del bitllet per predir si un passatger va sobreviure o no

def classificador_preu_bitllet(df, preu_minim=20):
    """
    Classifica els passatgers en funció del preu del bitllet
    El plantejament és que els passatgers que van pagar més tenien més probabilitats de sobreviure.
    El valor per defecte és 20, però podem canviar-lo
    """
    return (df['fare'] > preu_minim).astype(int)


model = FunctionClassifier(classificador_preu_bitllet)

# Preparem el classificador
model.fit(X_train, y_train)

### Evaluem el classificador

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, model.predict(X_test)))

El rendiment del classificador és prou dolent. Segurament, el preu del bitllet no és una bona variable per predir si un passatger va sobreviure o no. Així i tot intentarem millorar el seu rendiment afinant el paràmetre preu_minim.

Utilitzarem GridSearchCV per trobar el millor valor per preu_minim. GridSearchCV ens permet fer una cerca en graella per trobar els millors paràmetres, la puntuació vindrà donada per la precisió (accuracy), la precisió (precision) i la sensibilitat (recall).

**Accuracy**: Quantes prediccions són correctes?
**Precision**: Quantes prediccions positives són correctes?
**Recall**: Quantes prediccions positives són correctes de totes les positives?

In [ ]:
import numpy as np
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

# Creem un classificador amb un preu mínim de 20
mod = FunctionClassifier(classificador_preu_bitllet, preu_minim=20)

# L'objecte GridSearchCV ens permet fer una cerca en graella per trobar els millors paràmetres.
# En aquest cas, buscarem el millor preu mínim per classificar els passatgers
grid = GridSearchCV(mod,
                    cv=2,
                    param_grid={'preu_minim': np.linspace(0, 100, 30)},
                    scoring={'accuracy': make_scorer(accuracy_score),
                             'precision': make_scorer(precision_score),
                             'recall': make_scorer(recall_score)},
                    refit='accuracy')
grid.fit(X_train, y_train)

print(classification_report(y_test, grid.predict(X_test)))

In [ ]:
# Podem veure quin ha estat el millor preu mínim
grid.best_params_

### Exploració interactiva

Ara que ja hem vist que el preu del bitllet no és una bona variable per predir si un passatger va sobreviure o no, podem explorar les dades per trobar una variable millor.

Per fer-ho, utilitzarem la funció InteractiveCharts. Aquesta funció ens permetrà explorar les dades de forma interactiva. En aquest cas, utilitzarem la funció parallel_coordinates per veure com es distribueixen les dades en funció de les variables explicatives.

In [ ]:
from hulearn.experimental.interactive import parallel_coordinates

parallel_coordinates(df, label="survived", height=200)

Partint de la hipótesi que les dones i els nens van tenir més probabilitats de sobreviure, podem explorar les dades per veure si es veritat i sembla que  sí (comptant amb les diferències de classe social): Si Solament deixem les dones i els nens de 1a i 2a classe, la majoria van sobreviure.

Així doncs, podem crear un classificador que utilitzi el sexe, la edat i la classe del passatger per predir si va sobreviure o no.

In [ ]:
def classificador_edat_sexe_classe(df, edat=12):
    """
    Classifica els passatgers en funció del sexe, la edat i la classe del passatger
    El plantejament és que les dones i els nens van tenir més probabilitats de sobreviure
    """
    regla_dones = (df['pclass'] < 3.0) & (df['sex'] == "female")
    regla_nens = (df['pclass'] < 3.0) & (df['age'] <= edat)
    return regla_dones | regla_nens


mod = FunctionClassifier(classificador_edat_sexe_classe)
mod.fit(X_train, y_train)
mod.score(X_test, y_test)

In [ ]:
grid = GridSearchCV(mod,
                    cv=2,
                    param_grid={'edat': np.linspace(0, 50, 50)},
                    scoring={'accuracy': make_scorer(accuracy_score),
                             'precision': make_scorer(precision_score),
                             'recall': make_scorer(recall_score)},
                    refit='accuracy')
grid.fit(X_train, y_train)

print(grid.best_params_)
predict = grid.predict(X_train)

print(predict)
print(classification_report(y_test, grid.predict(X_test)))

Els nombres son prou millors que els anteriors. Així doncs, podem utilitzar aquest classificador per fer prediccions sobre nous passatgers.

## FIGS

FIGS és una llibreria que ens permet crear regles per classificar les dades. En aquest cas, utilitzarem les dades del Titanic per crear regles que ens permetin predir si un passatger va sobreviure o no.

In [ ]:
# instal·lem la llibreria
%pip install imodels

In [ ]:
# Carreguem la llibreria
from imodels import FIGSClassifier

In [ ]:
# Preparem les dades
X_skope = df.drop(columns=['name', 'sibsp'])
X_skope['sex'] = X_skope['sex'].replace(['female', 'male'], [0, 1])

X_train, X_test, y_train, y_test = train_test_split(
    X_skope.drop(['survived'], axis=1), X_skope['survived'], test_size=0.25, random_state=42)

In [ ]:
# Creem el classificador
clf = FIGSClassifier(max_rules=4)
# Entrenem el classificador
clf.fit(X_train, y_train)

In [ ]:
# Podem veure el rendiment del classificador
print(classification_report(y_test, clf.predict(X_test)))

Podem observar que les regles que ha creat el classificador tenen un rendiment semblant al classificador que hem creat amb Human learn i els conceptes que hem creat amb Human learn són més fàcils d'interpretar i podem seguir afegint regles per millorar el rendiment del classificador (per exemple, podríem incorporar la variable `fare`).